In [1]:
from qol.mesa.launcher import *

from qol.mesa.presets.make_merger_MS_HeWD import *

import qol.info as info

In [2]:
run_path = '/Users/nrui/Desktop/RG_HeWD_test/'

MWD_in_Msun = 0.15 # test for now -- ZJ+13 have 0.4 #0.4 # HeWD mass
Mcore_in_Msun = 0.2 # RG He core mass
Menv_in_Msun = 1.4
T_WD = 10e3 # temperature of WD
net_name = 'pp_cno_extras_o18_ne22.net' #'cno_extras_o18_to_mg26.net', # net
ringdown_time_yr = 1e6 # ringdown timescale to HSE
enable_pgstar = True
rgb_wind = True
alpha_semiconvection = 0. #4e-2, # semiconvection
thermohaline_coeff = 2. # thermohaline -- probably more important
source_sdk = True # manually activate sdk, since Caltech HPC doesn't seem to like it
mesh_delta_coeff = 1.

In [3]:
# generate tasks

# these three are the same as in the MS+HeWD case
task_evolve_rg = helper_merger_MS_HeWD_evolve_rg(enable_pgstar=enable_pgstar, net_name=net_name, MWD_in_Msun=MWD_in_Msun, mesh_delta_coeff=mesh_delta_coeff)
task_strip_rg = helper_merger_MS_HeWD_strip_rg(enable_pgstar=enable_pgstar, MWD_in_Msun=MWD_in_Msun, mesh_delta_coeff=mesh_delta_coeff)
task_cool_he_wd = helper_merger_MS_HeWD_cool_he_wd(enable_pgstar=enable_pgstar, T_WD=T_WD, mesh_delta_coeff=mesh_delta_coeff)

# substituted tasks here

task_env_to_th_eq = helper_merger_MS_HeWD_env_to_th_eq(enable_pgstar=enable_pgstar, net_name=net_name, MMS_in_Msun=Menv_in_Msun, mesh_delta_coeff=mesh_delta_coeff)

# substituted tasks here

task_remnant_ringdown = helper_merger_MS_HeWD_remnant_ringdown(enable_pgstar=enable_pgstar, ringdown_time_yr=ringdown_time_yr, mesh_delta_coeff=mesh_delta_coeff)
task_remnant_to_trgb = helper_merger_MS_HeWD_remnant_to_trgb(enable_pgstar=enable_pgstar, rgb_wind=rgb_wind, mesh_delta_coeff=mesh_delta_coeff)
task_trgb_to_zacheb = helper_merger_MS_HeWD_trgb_to_zacheb(enable_pgstar=enable_pgstar, rgb_wind=rgb_wind, mesh_delta_coeff=mesh_delta_coeff)
task_zacheb_to_co_wd = helper_merger_MS_HeWD_zacheb_to_co_wd(enable_pgstar=enable_pgstar, mesh_delta_coeff=mesh_delta_coeff)
task_cool_co_wd_early = helper_merger_MS_HeWD_cool_co_wd_early(enable_pgstar=enable_pgstar, alpha_semiconvection=alpha_semiconvection, thermohaline_coeff=thermohaline_coeff, mesh_delta_coeff=mesh_delta_coeff)
task_cool_co_wd_late = helper_merger_MS_HeWD_cool_co_wd_late(enable_pgstar=enable_pgstar, alpha_semiconvection=alpha_semiconvection, thermohaline_coeff=thermohaline_coeff, mesh_delta_coeff=mesh_delta_coeff)


In [4]:
# Accrete helium mass onto it
inlist = MesaInlist(name='subduct_he_wd')
if enable_pgstar:
    inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/subduct_he_wd/')
inlist.use_qol_pgstar()

inlist.load_model('cool_he_wd.mod')
inlist.set_Zbase(0.02)

inlist.gain_mass(max_star_mass_for_gain=MWD_in_Msun+Mcore_in_Msun, mass_change=1e-3,
              accrete_same_as_surface=False, accretion_h1=0., accretion_h2=0., accretion_he3=0., accretion_he4=0.98, accretion_zfracs=3)
inlist.disable_nuclear_burning()
inlist.disable_mixing()
### assume core mass is < 1 Msun lol
inlist.reset_age() # need to do this to make max_age work
inlist.max_age(1e3)

### define thresholds relevant to composition
inlist.surface_avg_abundance_dq(1e-2)
inlist.he_core_boundary_h1_fraction(1e-3)

inlist.mesh_delta_coeff(mesh_delta_coeff)

inlist.save_final_model('subduct_he_wd.mod')

task_subduct_he_wd = inlist

In [5]:
# Generate envelope boundary conditions
script = MesaPythonScript(name='inner_bc',
        template=f'{info.qol_path}mesa/templates/scripts/call_create_env_inlist_from_core.py',
            const_args=[Menv_in_Msun], prereqs=['subduct_he_wd.mod'], products=['inlist_env_inner_bc'])

task_inner_bc = script

In [6]:
script = MesaPythonScript(name='merge',
        template=f'{info.qol_path}mesa/templates/scripts/call_create_shell_burning_remnant.py',
        prereqs=['subduct_he_wd.mod', 'env_th_eq.mod'],
        products=['remnant_init.mod'])

task_merge = script

In [7]:
# create and save work directory
work = MesaWorkingDirectory(run_path=run_path)
work.copy_history_columns_list(f'{info.qol_path}mesa/resources/r24.08.1/history_columns_hewd_ms.list')
work.copy_profile_columns_list(f'{info.qol_path}mesa/resources/r24.08.1/profile_columns_qol.list')
work.load_qol_pgstar()

work.add_task(task_evolve_rg)
work.add_task(task_strip_rg)
work.add_task(task_cool_he_wd)
work.add_task(task_subduct_he_wd) ## new
work.add_task(task_inner_bc) ## new
work.add_task(task_env_to_th_eq)
work.add_task(task_merge) ## new
work.add_task(task_remnant_ringdown)
work.add_task(task_remnant_to_trgb)
work.add_task(task_trgb_to_zacheb)
work.add_task(task_zacheb_to_co_wd)
work.add_task(task_cool_co_wd_early)
work.add_task(task_cool_co_wd_late)

work.save_directory(grant_perms=True, source_sdk=source_sdk)
